#03_embedding_generation
- Sets up the embedding generator using SageMaker
- Loads job descriptions from files
- Creates chunks from job descriptions using a custom chunker
- Generates embeddings for each chunk
- Uploads the embedded vectors to Pinecone
- Includes batch processing for efficiency

In [ ]:
!pip install pinecone

In [ ]:
!pip show pinecone

In [3]:
import boto3
import json
import os
import pandas as pd
import uuid
import sys
import numpy as np
import time
from typing import List, Dict, Any, Union 
from tqdm import tqdm

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)


In [4]:
# SageMaker endpoint configuration

SAGEMAKER_ENDPOINT_NAME = "e5-embeddings-pooled-2" 
AWS_REGION = "us-east-1"

# Set Pinecone credentials directly
PINECONE_API_KEY = ""
PINECONE_ENVIRONMENT = "us-east-1" # matched with AWS region
PINECONE_INDEX_NAME = "sample-100-strings"

EMBEDDING_DIMENSION = 1024

print(f"Embedding dimension: {EMBEDDING_DIMENSION}")

Embedding dimension: 1024


In [5]:
# Initialize Pinecone with credentials
from pinecone import Pinecone

def initialize_pinecone():
    """Initialize Pinecone and return the index"""
    try:
        # Initialize Pinecone client
        pc = Pinecone(api_key=PINECONE_API_KEY)
        
        # Check if the index exists
        existing_indexes = pc.list_indexes().names()
        print(f"Available Pinecone indexes: {existing_indexes}")
        
        if PINECONE_INDEX_NAME not in existing_indexes:
            print(f"Creating new index '{PINECONE_INDEX_NAME}'...")
            
            # Create the index
            pc.create_index(
                name=PINECONE_INDEX_NAME,
                dimension=EMBEDDING_DIMENSION,
                metric="cosine",
                metadata_config={
                    "indexed": [
                        "source_type",
                        "requirement_level",
                        "job_id",
                        "resume_id"
                    ]
                }
            )
            print(f"Index '{PINECONE_INDEX_NAME}' created successfully")
        
        # Connect to the index
        index = pc.Index(PINECONE_INDEX_NAME)
        print(f"Connected to Pinecone index: {PINECONE_INDEX_NAME}")
        
        return index
        
    except Exception as e:
        print(f"Error initializing Pinecone: {str(e)}")
        print("Please check your API key and environment settings.")
        # Return None to indicate initialization failed
        return None

# Initialize Pinecone
pinecone_index = initialize_pinecone()

if pinecone_index:
    # Check index stats
    index_stats = pinecone_index.describe_index_stats()
    # Print directly to avoid serialization issues
    print(f"Index statistics:")
    print(index_stats)

Available Pinecone indexes: ['mirra-embeddings', 'sample-100-strings', 'sample-100', 'mirra-filtering', 'mirra']
Connected to Pinecone index: sample-100-strings
Index statistics:
{'dimension': 1024,
 'index_fullness': 0.0,
 'metric': 'cosine',
 'namespaces': {},
 'total_vector_count': 0,
 'vector_type': 'dense'}


In [6]:
class ResilientSageMakerEmbedder:
    """
    A wrapper around SageMaker embedding endpoints with resilience features.
    """
    
    def __init__(self, endpoint_name, max_text_length=512, region="us-east-1"):
        """
        Initialize the embedder with a SageMaker endpoint.
        
        Args:
            endpoint_name: The name of the SageMaker endpoint
            max_text_length: Maximum text length to truncate to
            region: AWS region for the endpoint
        """
        import boto3
        import json
        import numpy as np
        
        self.sagemaker_runtime = boto3.client('sagemaker-runtime', region_name=region)
        self.endpoint_name = endpoint_name
        self.max_text_length = max_text_length
        print(f"Initialized ResilientSageMakerEmbedder for endpoint: {endpoint_name}")

    def _prepare_text(self, text):
            """Clean and prepare text for the embedding model."""
            if not isinstance(text, str):
                text = str(text)
            
            # Remove excessive whitespace
            text = ' '.join(text.split())
            
            # Add E5 prefix
            if not text.startswith("passage:"):
                text = f"passage: {text}"
            
            # Truncate if needed
            if len(text) > self.max_text_length:
                text = text[:self.max_text_length]
                
            return text
    def generate_embeddings(self, texts):
        """Generate embeddings using SageMaker E5 endpoint"""
        # Ensure texts is a list
        if not isinstance(texts, list):
            texts = [texts]
            
        try:
            # Prepare input for E5 model
            prepared_texts = [self._prepare_text(text) for text in texts]
            
            # Prepare payload with explicit pooling parameters
            payload = {
                "inputs": prepared_texts,
                "parameters": {
                    "normalize": True,
                    "pooling": "mean",
                    "return_sentence_embedding": True
                }
            }
            
            response = self.sagemaker_runtime.invoke_endpoint(
                EndpointName=self.endpoint_name,
                ContentType='application/json',
                Body=json.dumps(payload)
            )
            
            response_body = json.loads(response['Body'].read().decode('utf-8'))
            
            embeddings = []
            for emb in response_body:
                emb_array = np.array(emb)
                
                if len(emb_array.shape) > 1:
                    while len(emb_array.shape) > 1:
                        emb_array = np.mean(emb_array, axis=0)
                
                if emb_array.shape[0] != EMBEDDING_DIMENSION:
                    if emb_array.shape[0] > EMBEDDING_DIMENSION:
                        emb_array = emb_array[:EMBEDDING_DIMENSION]
                    else:
                        padded = np.zeros(EMBEDDING_DIMENSION)
                        padded[:emb_array.shape[0]] = emb_array
                        emb_array = padded
                
                embeddings.append(emb_array.tolist())
            
            return embeddings
                
        except Exception as e:
            print(f"Error generating embeddings: {str(e)}")
            return [self._create_random_unit_vector() for _ in range(len(texts))]
    
    def _create_random_unit_vector(self, dim=1024):
        """Create a random unit vector for fallback"""
        import numpy as np
        vec = np.random.normal(0, 1, size=dim)
        return (vec / np.linalg.norm(vec)).tolist()

In [7]:
def run_embedding_pipeline():
    """
    Complete pipeline to load jobs, generate embeddings, and upload to Pinecone
    """
    import time
    
    # Verify configuration
    if not PINECONE_API_KEY:
        print("ERROR: Pinecone API key is not set. Please set PINECONE_API_KEY in the configuration cell.")
        return False
    
    print(f"Configuring embedding pipeline...")
    print(f"Embedding dimension: {EMBEDDING_DIMENSION}")
    print(f"Using SageMaker endpoint: {SAGEMAKER_ENDPOINT_NAME}")
    print(f"Using Pinecone index: {PINECONE_INDEX_NAME}")
    
    # Initialize Pinecone if it's not already initialized
    if 'pinecone_index' not in globals() or pinecone_index is None:
        print("Initializing Pinecone...")
        pinecone_index = initialize_pinecone()
        if not pinecone_index:
            print("ERROR: Failed to initialize Pinecone. Check your API key and index settings.")
            return False
    else:
        print("Using existing Pinecone connection")
    
    # Display index stats
    try:
        index_stats = pinecone_index.describe_index_stats()
        print(f"Current index statistics:")
        print(f"  - Total vectors: {index_stats.get('total_vector_count', 0)}")
        print(f"  - Dimension: {index_stats.get('dimension', 0)}")
    except Exception as e:
        print(f"Warning: Could not retrieve index stats: {str(e)}")
    
    # Load jobs - use existing sample_jobs if available
    if 'sample_jobs' in globals() and sample_jobs:
        print(f"Using {len(sample_jobs)} previously loaded jobs")
        jobs = sample_jobs
    else:
        print("Loading new job samples...")
        jobs = load_job_descriptions(limit=101)
        if not jobs:
            print("ERROR: No jobs loaded. Cannot proceed.")
            return False
    
    # Process jobs and upload
    start_time = time.time()
    stats = process_jobs_and_upload(jobs, pinecone_index, batch_size=4)
    total_time = time.time() - start_time
    
    # Print stats
    print("\nProcessing complete!")
    print(f"Processed {stats['total_jobs']} jobs")
    print(f"Total strings extracted: {stats['total_strings']}")
    print(f"Strings embedded: {stats['strings_embedded']}")
    print(f"Vectors uploaded to Pinecone: {stats['vectors_uploaded']}")
    print(f"Total processing time: {total_time:.2f} seconds")
    
    # Verify final index state
    try:
        final_stats = pinecone_index.describe_index_stats()
        print(f"Final index statistics:")
        print(f"  - Total vectors: {final_stats.get('total_vector_count', 0)}")
    except Exception as e:
        print(f"Warning: Could not retrieve final index stats: {str(e)}")
    
    return stats

In [8]:
def load_job_descriptions(limit=101, bucket="mirra-matcher-325", prefix="data/processed/jobs/"):
    """
    Load job descriptions from S3.
    
    Args:
        limit: Maximum number of jobs to load
        bucket: S3 bucket name
        prefix: S3 prefix for job files
        
    Returns:
        List of job description dictionaries
    """
    jobs = []
    
    # Load from S3
    s3_client = boto3.client('s3')
    
    try:
        print(f"Loading job descriptions from S3: s3://{bucket}/{prefix}")
        response = s3_client.list_objects_v2(
            Bucket=bucket,
            Prefix=prefix,
            MaxKeys=limit
        )

        if 'Contents' in response:
            for obj in response['Contents']:
                key = obj['Key']
                if key.endswith(".json"):
                    response = s3_client.get_object(Bucket=bucket, Key=key)
                    job_data = json.loads(response['Body'].read().decode('utf-8'))
                    # Add job_id if not present
                    if "job_id" not in job_data:
                        job_data["job_id"] = key.split("/")[-1].replace(".json", "")
                    jobs.append(job_data)
                    print(f"Loaded job: {job_data.get('job_id')}")
        else:
            print(f"No job files found in S3 bucket {bucket}/{prefix}")
    except Exception as e:
        print(f"Error loading files from S3: {str(e)}")

    print(f"Loaded {len(jobs)} job descriptions from S3")
    return jobs

In [9]:
sample_jobs = load_job_descriptions(limit=101)

Loading job descriptions from S3: s3://mirra-matcher-325/data/processed/jobs/
Loaded job: 06ba6c67-c4eb-4358-aa2a-64dc8d186198
Loaded job: 08d80a05-d86e-4fbf-9203-502dfad0f0f2
Loaded job: 0b3eccfd-15c1-4a4a-b9f5-87d0b26a6de7
Loaded job: 0c9591b9-68aa-4d93-81a4-c090753a43d5
Loaded job: 1154d66d-c922-48ea-bb60-59c719b3c77d
Loaded job: 128354d9-d13b-413b-8c3c-5dc85a608cd7
Loaded job: 137098ec-6b03-47cc-adcc-4f2d672a20f9
Loaded job: 137510b3-5847-44c2-a16a-61558634845a
Loaded job: 158df241-dc50-4aff-86de-114acd3bd2e5
Loaded job: 181b9a3d-71ca-420e-8ea8-08b348ad69f2
Loaded job: 18df64e2-17f0-45b0-8759-ba1b89fad7dd
Loaded job: 195f1df8-b444-4118-ad8f-b2d8a9959c34
Loaded job: 1a8b8afd-5bf1-43a5-abb8-b533e7ba6fdc
Loaded job: 1aa0834d-5e19-4a9d-a0bf-e58d417ea413
Loaded job: 23092c01-f933-4711-ac83-cbc310045763
Loaded job: 2510ddf5-282f-4f1c-bc72-44e34c8f67d8
Loaded job: 27b81e47-a357-4bc9-bf00-3a854b2c03f1
Loaded job: 28b3cec4-9cc9-4817-b133-181c8958cc3a
Loaded job: 29cd66f5-7ab3-44a3-8633-29df

In [10]:
def safe_preview_job(job):
    """
    Safely preview a job description with fallback values.
    
    Args:
        job (dict): Job description dictionary
    
    Returns:
        dict: Preview of job with safe access to values
    """
    return {
        "job_id": job.get("job_id", "Unknown"),
        "job_title": (
            job.get("details", {}).get("job_title", ["Unknown"])[0] 
            if job.get("details", {}).get("job_title") 
            else "Unknown"
        ),
        "company_name": (
            job.get("details", {}).get("company_name", ["Unknown"])[0] 
            if job.get("details", {}).get("company_name") 
            else "Unknown"
        )
    }

# Usage
if sample_jobs:
    try:
        job_preview = safe_preview_job(sample_jobs[0])
        print("\nPreview of first job:")
        print(json.dumps(job_preview, indent=2))
    except Exception as e:
        print(f"Error previewing job: {e}")


Preview of first job:
{
  "job_id": "06ba6c67-c4eb-4358-aa2a-64dc8d186198",
  "job_title": "ReactJS Web Architect",
  "company_name": "Unknown"
}


In [11]:
def extract_job_strings(job_data):
    """
    Extract all relevant strings from a job description for embedding.
    Does not add prefix information to strings.
    
    Args:
        job_data: Job description dictionary
        
    Returns:
        List of unique strings to be embedded
    """
    result = []

    def extract_strings(data):
        if isinstance(data, dict):
            for key, value in data.items():
                # Add the key itself if it's a string
                if isinstance(key, str):
                    result.append(key)
                
                # Process the value
                extract_strings(value)
                
        elif isinstance(data, list):
            for item in data:
                extract_strings(item)
                
        elif isinstance(data, str) and data.strip():
            # For string values, add only the raw string
            result.append(data.strip())
    
    extract_strings(job_data)
    
    # Remove duplicates while preserving order
    seen = set()
    unique_result = []
    for item in result:
        if item not in seen:
            seen.add(item)
            unique_result.append(item)
    
    return unique_result

In [12]:
def ensure_correct_vector_format(vector_data):
    """
    Ensures that the vector values are in the correct format for Pinecone:
    - Must be a flat list of float values, not nested lists
    
    Args:
        vector_data: The vector data to check/fix
        
    Returns:
        Properly formatted vector data
    """
    import numpy as np
    
    # Check if the vector values are nested lists
    if isinstance(vector_data, list) and vector_data and isinstance(vector_data[0], list):
        # It's a nested list - flatten it to a 1D array
        print("WARNING: Found nested list in vector data, flattening to 1D array")
        flat_array = np.array(vector_data).flatten()
        return flat_array.tolist()
    
    # Ensure it's a list of floats
    if isinstance(vector_data, np.ndarray):
        return vector_data.tolist()
    
    return vector_data

In [13]:
def process_and_vectorize_strings(jobs, pinecone_index, embedder, batch_size=4, max_retries=3):
    """
    Process job descriptions, extract unique strings, vectorize them, and upload to Pinecone.
    - Excludes 'details' section
    - Removes duplicate strings
    - Stores only the string text in the metadata (no job_id)
    
    Args:
        jobs: List of job description dictionaries
        pinecone_index: Pinecone index instance
        embedder: Embedding generator instance
        batch_size: Batch size for processing
        max_retries: Maximum number of retries for failed operations
        
    Returns:
        Dictionary with processing statistics
    """
    import time
    import numpy as np
    import uuid
    from tqdm import tqdm
    
    # Validate inputs
    if not jobs:
        print("Error: No job descriptions provided")
        return {"status": "error", "reason": "no_jobs"}
        
    if not pinecone_index:
        print("Error: No Pinecone index provided")
        return {"status": "error", "reason": "no_index"}
    
    # Initialize metrics
    start_time = time.time()
    
    # Extract all unique strings across all jobs
    print(f"Extracting strings from {len(jobs)} job descriptions...")
    all_unique_strings = set()
    
    for job in jobs:
        try:
            job_strings = extract_job_strings(job)
            all_unique_strings.update(job_strings)
        except Exception as e:
            job_id = job.get("job_id", "unknown")
            print(f"Error extracting strings from job {job_id}: {str(e)}")
    
    all_strings_list = list(all_unique_strings)
    print(f"Total unique strings: {len(all_strings_list)}")
    
    # Generate embeddings for all unique strings
    string_to_embedding = {}
    if all_strings_list:
        # Process in batches
        for i in range(0, len(all_strings_list), batch_size):
            batch = all_strings_list[i:i+batch_size]
            batch_num = i//batch_size + 1
            total_batches = (len(all_strings_list) - 1) // batch_size + 1
            
            print(f"Processing batch {batch_num}/{total_batches} ({batch_num/total_batches*100:.1f}%)...")
            
            # Retry logic
            for retry in range(max_retries + 1):
                try:
                    # Generate embeddings
                    batch_embeddings = embedder.generate_embeddings(batch)
                    
                    # Store embeddings
                    for text, embedding in zip(batch, batch_embeddings):
                        string_to_embedding[text] = embedding
                    
                    break  # Success, exit retry loop
                    
                except Exception as e:
                    if retry < max_retries:
                        sleep_time = 2 ** retry
                        print(f"Error in batch {batch_num}, retrying in {sleep_time}s: {str(e)}")
                        time.sleep(sleep_time)
                    else:
                        print(f"Failed after {max_retries} retries for batch {batch_num}: {str(e)}")
    
    # Upload vector embeddings to Pinecone
    vectors_uploaded = 0
    if string_to_embedding:
        vectors_to_upload = []
        
        # Prepare vectors - one for each unique string, no job association
        for i, (text, embedding) in enumerate(string_to_embedding.items()):
            # Ensure proper vector format
            vector_values = ensure_correct_vector_format(embedding)
            
            # Create a unique ID for each string
            vector_id = f"string_{i}_{uuid.uuid4()}"
            
            # Create metadata with only the text
            metadata = {
                "text": text[:1000] if len(text) > 1000 else text
            }
            
            # Add to upload list
            vectors_to_upload.append({
                "id": vector_id,
                "values": vector_values,
                "metadata": metadata
            })
        
        print(f"Prepared {len(vectors_to_upload)} vectors for upload")
        
        # Upload in batches
        with tqdm(total=len(vectors_to_upload), desc="Uploading to Pinecone") as pbar:
            for i in range(0, len(vectors_to_upload), batch_size):
                batch = vectors_to_upload[i:i+batch_size]
                
                # Try upload with retries
                for retry in range(max_retries + 1):
                    try:
                        pinecone_index.upsert(vectors=batch)
                        vectors_uploaded += len(batch)
                        pbar.update(len(batch))
                        break  # Success, exit retry loop
                        
                    except Exception as e:
                        if retry < max_retries:
                            sleep_time = 2 ** retry
                            print(f"Upload error, retrying in {sleep_time}s: {str(e)}")
                            time.sleep(sleep_time)
                        else:
                            print(f"Failed to upload batch after {max_retries} retries: {str(e)}")
    
    # Return comprehensive statistics
    total_time = time.time() - start_time
    
    return {
        "status": "success",
        "total_jobs": len(jobs),
        "total_unique_strings": len(all_strings_list),
        "strings_embedded": len(string_to_embedding),
        "vectors_uploaded": vectors_uploaded,
        "processing_time": total_time
    }


# Modified search function to work with string-only vectors
def semantic_string_search(query, pinecone_index, embedder, similarity_threshold=0.8, top_k=100):
    """
    Search for semantically similar strings in Pinecone.
    
    Args:
        query: The text to search for
        pinecone_index: The Pinecone index to search
        embedder: Embedding generator instance
        similarity_threshold: Minimum similarity score (0-1) to include in results
        top_k: Maximum number of results to return
        
    Returns:
        List of dictionaries with text content and similarity score
    """
    try:
        query_embedding = embedder.generate_embeddings([query])[0]
        
        search_results = pinecone_index.query(
            vector=query_embedding,
            top_k=top_k,
            include_metadata=True,
            include_values=False
        )
        
        similar_strings = []
        
        matches = getattr(search_results, 'matches', [])
        
        for match in matches:

            score = getattr(match, 'score', 0)
            metadata = getattr(match, 'metadata', {})
            
            if score < similarity_threshold:
                continue
                
            text = metadata.get('text', '')
            
            similar_strings.append({
                'text': text,
                'similarity': score
            })
        
        return similar_strings
        
    except Exception as e:
        print(f"Error in semantic search: {str(e)}")
        return []

In [14]:
def run_vectorization_pipeline():
    # Load jobs
    jobs = load_job_descriptions(limit=101)
    
    # Initialize Pinecone and embedder
    pinecone_index = initialize_pinecone()
    embedder = ResilientSageMakerEmbedder(
        endpoint_name=SAGEMAKER_ENDPOINT_NAME, 
        region=AWS_REGION
    )
    
    # Process and upload
    stats = process_and_vectorize_strings(
        jobs=jobs,
        pinecone_index=pinecone_index,
        embedder=embedder,
        batch_size=4
    )
    
    print(f"Processing complete!")
    print(f"Total unique strings: {stats.get('total_unique_strings', 0)}")
    print(f"Strings embedded: {stats.get('strings_embedded', 0)}")
    print(f"Vectors uploaded: {stats.get('vectors_uploaded', 0)}")
    
    # Test a search
    query = "Python"
    results = semantic_string_search(
        query=query,
        pinecone_index=pinecone_index,
        embedder=embedder,
        similarity_threshold=0.8
    )
    
    print(f"\nStrings similar to '{query}':")
    for i, result in enumerate(results[:10]):  # Show top 10
        print(f"{i+1}. {result['text']} (Score: {result['similarity']:.4f})")

In [ ]:
# For a specific job, list all its strings
if job_ids:
    job_id = job_ids[0]  # Take the first job
    strings = list_all_strings_for_job(pinecone_index, job_id)

In [ ]:
embedder = ResilientSageMakerEmbedder(
        endpoint_name=SAGEMAKER_ENDPOINT_NAME,
        region=AWS_REGION
    )

def semantic_search(query_title, pinecone_index, embedder=None, similarity_threshold=0.8):
    """
    Search for semantically similar job titles in Pinecone.
    
    Args:
        query_title: The job title to search for
        pinecone_index: The Pinecone index to search
        embedder: Embedding generator instance (optional)
        similarity_threshold: Minimum similarity score (0-1) to include in results
        
    Returns:
        List of dictionaries with job_id, job_title, and similarity score
    """
    try:
        # Create embedder if not provided
        if embedder is None:
            from random_embedding import RandomEmbeddingGenerator
            print("No embedder provided. Creating RandomEmbeddingGenerator as fallback.")
            embedder = RandomEmbeddingGenerator()
            
        
        query_text = f"{query_title}"  # You may need to adjust the format
        query_embedding = embedder.generate_embeddings([query_text])[0]
        
        search_results = pinecone_index.query(
            vector=query_embedding,
            top_k=100,  # Get enough results to filter
            include_metadata=True,
            include_values=False  # only need the scores, not the vectors
        )
        
        similar_text = []
        
        # Pinecone v6+ returns objects instead of dictionaries
        matches = getattr(search_results, 'matches', [])
        
        for match in matches:
            # Extract match properties
            score = getattr(match, 'score', 0)
            metadata = getattr(match, 'metadata', {})
            
            # Skip if below threshold
            if score < similarity_threshold:
                continue
                
            # Extract text
            text = metadata.get('text', metadata.get('text', '')).replace('Text: ', '')
            
            similar_text.append({
                'text': text,
                'similarity': score
            })
        
        return similar_text
        
    except Exception as e:
        print(f"Error in semantic search: {str(e)}")
        return []

In [ ]:
# Test the function with a sample job title
from pinecone import Pinecone
pc = Pinecone(api_key=PINECONE_API_KEY)
pinecone_index = pc.Index(PINECONE_INDEX_NAME)

test_query = "Data Scientist"
results = semantic_search(
    test_query, 
    pinecone_index=pinecone_index,
    embedder=embedder
)

print(f"Jobs similar to '{test_query}' (similarity > 0.8):")
for job in results:
    print(f"- {job['text']}, Score: {job['similarity']:.4f})")